In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=15

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=15


In [3]:
import sys
sys.path.append('..')

import json
import os
import cv2
import numpy as np
import torch
from PIL import Image
from pytorch_lightning import seed_everything
from glob import glob
import matplotlib.pyplot as plt

from cldm.model import create_model, load_state_dict
from dataset import PhotoSketchDataset
from inference import run_sampler
from share import *

/raid/lingo/grandg/miniconda3/envs/control/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


logging improved.


In [4]:
model = create_model('../models/cldm_v15.yaml').cpu()

No module 'xformers'. Proceeding without it.
ControlLDM: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Loaded model config from [../models/cldm_v15.yaml]


In [5]:
# CKPT_PATH = "/raid/lingo/alexisro/ControlNet/project/experiments/sketch_0503/lightning_logs/version_9/checkpoints/epoch=28-step=57999.ckpt"
CKPT_PATH = "/raid/lingo/alexisro/ControlNet/project/experiments/cropped/from_finetuned_patience_20/lr=1e-06_bs=4/lightning_logs/version_0/checkpoints/epoch=41-step=4199.ckpt"

model.load_state_dict(load_state_dict(CKPT_PATH, location='cuda'))
model = model.cuda()

Loaded state_dict from [/raid/lingo/alexisro/ControlNet/project/experiments/cropped/from_finetuned_patience_20/lr=1e-06_bs=4/lightning_logs/version_0/checkpoints/epoch=41-step=4199.ckpt]


In [7]:
def display_images_in_grid(sketches, outputs, prompt, save_path=None):
    # Assuming you have two lists of image arrays called "sketches" and "outputs"
    num_images = len(sketches)

    # Calculate the number of rows needed to display the images in a grid
    num_rows = 2

    # Calculate the number of columns needed to display the images in a grid
    num_cols = len(sketches)

    # Create a new figure
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(num_cols*2, num_rows*2), gridspec_kw={'wspace': 0., 'hspace': 0.})

    # Loop through the sketch and output image arrays and display each one in a subplot
    for i, (sketch, output) in enumerate(zip(sketches, outputs)):
        row = 0
        col = i

        axes[row, col].imshow(sketch)
        axes[row + 1, col].imshow(output)

        axes[row, col].axis('off')
        axes[row + 1, col].axis('off')

    plt.subplots_adjust(wspace=0., hspace=0.)
    plt.suptitle(prompt, fontsize=24)
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
    else:
        plt.show()

In [ ]:
DIR_FRAME_SEQUENCES = "data/frame_sequences/sketches/"
DIR_RENDERS_OUT = "data/frame_sequences/renders/"
os.makedirs(DIR_RENDERS_OUT, exist_ok=True)

DIR_CAPTIONS = "lavis/captions/"
DIR_SKETCH_GROUNDTRUTH = "data/sketch/"

N_FRAMES_PER_SEQUENCE = 8
IMAGE_RESOLUTION = 512
SEED = 42

render_results = []

for dir_frames in sorted(glob(os.path.join(DIR_FRAME_SEQUENCES, "*"))):
    print(dir_frames)
    svg_id = os.path.split(dir_frames)[-1][len("svg_"):len("svg_")+4]
    svg_version_id = os.path.split(dir_frames)[-1][len("svg_XXXX_"):len("svg_XXXX_")+2]
    
    # Get caption
    caption_path = os.path.join(DIR_CAPTIONS, f"{str(svg_id).zfill(8)}.json")    
    with open(caption_path, "r") as f:
        caption_text = json.load(f)[0]
        print(caption_text)
    
    frame_paths = sorted(glob(os.path.join(dir_frames, "*.png")))

    sketches, outputs = [], []
    
    frame_range = np.linspace(1, len(frame_paths) - 1, N_FRAMES_PER_SEQUENCE, endpoint=True, dtype=int)

    for i, idx in enumerate(frame_range):
        path = frame_paths[idx]
        
        sketch = cv2.imread(path)

        results = run_sampler(model, sketch, caption_text, seed=SEED, image_resolution=IMAGE_RESOLUTION, show_progress=False)
        sketches.append(sketch)
        outputs.append(results[0])
        
    render_results.append((svg_id, caption_text, sketches, outputs))
        
    save_path = os.path.join(DIR_RENDERS_OUT, f"sequence_{svg_id}_{svg_version_id}.png")
    display_images_in_grid(sketches, outputs, caption_text, save_path=save_path)

Global seed set to 42


data/frame_sequences/sketches/svg_0270_02_frames
a group of people walking on a beach
Data shape for DDIM sampling is (1, 4, 64, 96), eta 0.0
Running DDIM Sampling with 20 timesteps


Global seed set to 42


Data shape for DDIM sampling is (1, 4, 64, 96), eta 0.0
Running DDIM Sampling with 20 timesteps


Global seed set to 42


...

Data shape for DDIM sampling is (1, 4, 64, 88), eta 0.0
Running DDIM Sampling with 20 timesteps


Global seed set to 42


Data shape for DDIM sampling is (1, 4, 64, 88), eta 0.0
Running DDIM Sampling with 20 timesteps


Global seed set to 42


data/frame_sequences/sketches/svg_1000_02_frames
a woman holding a frisbee with a dog on a leash
Data shape for DDIM sampling is (1, 4, 96, 64), eta 0.0
Running DDIM Sampling with 20 timesteps


In [8]:
for svg_id, caption_text, sketches, outputs in render_results:        
    save_path = os.path.join(DIR_RENDERS_OUT, f"sequence_{svg_id}.png")
    # display_images_in_grid(sketches, outputs, caption_text)
    display_images_in_grid(sketches, outputs, caption_text, save_path=save_path)